In [130]:
!pip install xgboost
!pip install fuzzywuzzy pandas python-Levenshtein

   ---------------------------------------- 0.0/98.8 kB ? eta -:--:--
   ---------------------------------------- 98.8/98.8 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB 8.6 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.6 MB 7.2 MB/s eta 0:00:01
   ---------------------- ----------------- 0.9/1.6 MB 7.2 MB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.6 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.8 MB/s eta 0:00:00


In [108]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix
import plotly.figure_factory as ff
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
import re

In [136]:
from fuzzywuzzy import process


### Reading Data 

In [146]:
credit_features_df = pd.read_csv(r"E:\CODE\test\archive\credit_features_subset.csv")
loan_applications_df = pd.read_csv(r"E:\CODE\test\archive\loan_applications.csv")

### Print Top 3 Rows

In [149]:
credit_features_df.head(3)

,UID,ALL_AgeOfOldestAccount,ALL_AgeOfYoungestAccount,ALL_Count,ALL_CountActive,ALL_CountClosedLast12Months,ALL_CountDefaultAccounts,ALL_CountOpenedLast12Months,ALL_CountSettled,ALL_MeanAccountAge,ALL_SumCurrentOutstandingBal,ALL_SumCurrentOutstandingBalExcMtg,ALL_TimeSinceMostRecentDefault,ALL_WorstPaymentStatusActiveAccounts
0,216009,227,12,14,11,0,3,0,3,71.36,9292,9292,-1,7
1,217537,49,10,7,6,1,3,1,1,29.33,4592,4592,-1,7
2,221803,134,0,12,11,0,3,5,1,32.33,2737,2737,-1,7


In [151]:
loan_applications_df.head(3)

,UID,ApplicationDate,Amount,Term,EmploymentType,LoanPurpose,Success
0,4921736,03/07/2020,2000,60,Employed - full time,Unexpected bills,0
1,1241981,04/02/2020,3000,60,Employed - full time,Starting new bussniess,0
2,5751748,02/08/2020,20000,60,Employed - full time,Business capital,0


### Combine 

In [154]:
combined_df = pd.merge(loan_applications_df,credit_features_df , on='UID', how='inner') 

In [156]:
combined_df.head(3)

,UID,ApplicationDate,Amount,Term,EmploymentType,LoanPurpose,Success,ALL_AgeOfOldestAccount,ALL_AgeOfYoungestAccount,ALL_Count,ALL_CountActive,ALL_CountClosedLast12Months,ALL_CountDefaultAccounts,ALL_CountOpenedLast12Months,ALL_CountSettled,ALL_MeanAccountAge,ALL_SumCurrentOutstandingBal,ALL_SumCurrentOutstandingBalExcMtg,ALL_TimeSinceMostRecentDefault,ALL_WorstPaymentStatusActiveAccounts
0,4921736,03/07/2020,2000,60,Employed - full time,Unexpected bills,0,162,17,18,12,0,0,0,6,70.94,68555,15019,-1,0
1,1241981,04/02/2020,3000,60,Employed - full time,Starting new bussniess,0,266,30,14,10,0,4,0,4,104.79,2209,2209,-1,7
2,5751748,02/08/2020,20000,60,Employed - full time,Business capital,0,90,52,4,2,0,1,0,2,68.25,5108,5108,-1,7


### Create list of Reasons

In [210]:
list_of_reasons = combined_df["LoanPurpose"]

In [212]:
list_of_reasons.nunique()

7

2444 Reasons, but mostly they could be categories into major categories with keywords such as vehicle, wedding, home, debt, credit card etc...

In [215]:
keywords_to_category = {'car': 'vehicle','vehicle': 'vehicle','van': 'vehicle','motorcycle': 'vehicle',
    'wedding': 'wedding','credit card': 'credit card','debt': 'debt','consol': 'debt','pay': 'debt',
    'business': 'business','house': 'home','home': 'home','loan': 'debt','holid': 'personal'}

# Function to categorize loan purposes
def categorize_loan_purpose(purpose): 
    purpose_lower = purpose.lower()  # Convert to lowercase
    best_match = process.extractOne(purpose_lower, keywords_to_category.keys()) # Use fuzzy matching to find the best keyword match
    # Check if the best match score is above a threshold (e.g., 80)
    if best_match[1] >= 80:  # You can adjust this threshold as needed
        keyword = best_match[0]
        return keywords_to_category[keyword]  # Return the corresponding category
    
    return 'other'  # Default category if no match is found

# Apply the function to the DataFrame
combined_df['LoanPurpose'] = combined_df['LoanPurpose'].apply(categorize_loan_purpose)

In [216]:
combined_df['LoanPurpose'].nunique()

7

### Data Visualization

In [218]:
def plot_histogram(column):
    fig = px.histogram(combined_df, x=column)
    fig.update_layout( width = 700, height = 400)
    fig.show(renderer='iframe')

In [219]:
plot_histogram('LoanPurpose')

In [220]:
plot_histogram('EmploymentType')

In [221]:
combined_df

,UID,ApplicationDate,Amount,Term,EmploymentType,LoanPurpose,Success,ALL_AgeOfOldestAccount,ALL_AgeOfYoungestAccount,ALL_Count,ALL_CountActive,ALL_CountClosedLast12Months,ALL_CountDefaultAccounts,ALL_CountOpenedLast12Months,ALL_CountSettled,ALL_MeanAccountAge,ALL_SumCurrentOutstandingBal,ALL_SumCurrentOutstandingBalExcMtg,ALL_TimeSinceMostRecentDefault,ALL_WorstPaymentStatusActiveAccounts
0,4921736,03/07/2020,2000,60,Employed - full time,other,0,162,17,18,12,0,0,0,6,70.94,68555,15019,-1,0
1,1241981,04/02/2020,3000,60,Employed - full time,other,0,266,30,14,10,0,4,0,4,104.79,2209,2209,-1,7
2,5751748,02/08/2020,20000,60,Employed - full time,business,0,90,52,4,2,0,1,0,2,68.25,5108,5108,-1,7
3,7163425,23/09/2020,20000,60,Self employed,business,0,163,19,14,6,1,1,0,8,67.50,25738,25738,-1,0
4,227377,01/01/2020,5000,36,Employed - full time,vehicle,0,129,2,38,19,4,9,8,19,56.45,5801,5801,-1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8842,8681053,17/11/2020,5000,60,Employed - full time,wedding,0,164,2,22,16,2,0,5,6,52.36,10968,10968,-1,0
8843,8768469,20/11/2020,8000,24,Employed - full time,wedding,0,275,20,7,4,1,1,0,3,73.57,2204,2204,-1,7
8844,8794971,22/11/2020,1000,36,Employed - full time,wedding,0,145,8,14,13,0,0,1,1,44.14,6786,6786,-1,0
8845,8875138,24/11/2020,10000,60,Self employed,wedding,0,12,7,4,3,1,0,3,1,9.25,9398,9398,-1,0


In [222]:
plot_histogram('Term')

In [223]:
fig = px.histogram(combined_df, x="LoanPurpose", color="Success")
fig.show(renderer='iframe')

In [224]:
loan_success = combined_df.pivot_table(
    index='LoanPurpose', 
    columns='Success', 
    aggfunc='size', 
    fill_value = 0
).rename(columns={0: 'fail', 1: 'success'}).reset_index()

In [225]:
loan_success

Success,LoanPurpose,fail,success
0,business,2,0
1,credit card,131,16
2,debt,1027,114
3,home,1126,144
4,other,3497,381
5,vehicle,2075,261
6,wedding,67,6


In [226]:
fig = go.Figure(data=[go.Pie(labels=loan_success['LoanPurpose'], values=loan_success['success'], textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig.show(renderer='iframe')

In [227]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=combined_df['ALL_AgeOfOldestAccount'],
    nbinsx=30,
    marker=dict(color='#FFA07A', line=dict(color='black', width=1)),
    opacity=0.75,
    name="Age of Accounts"
))

fig.update_layout(
    title="Distribution of Age of Oldest Accounts",
    xaxis_title="Age of Oldest Account (in years)",
    yaxis_title="Frequency",
    bargap=0.1
)

fig.show(renderer='iframe')


### Preprocessing, Modelling and Prediction

In [241]:
combined_df = pd.get_dummies(combined_df, columns=['LoanPurpose'], dtype=int)

In [245]:
combined_df = pd.get_dummies(combined_df, columns=['EmploymentType'], dtype=int)

In [247]:
combined_df.head()

,UID,ApplicationDate,Amount,Term,Success,ALL_AgeOfOldestAccount,ALL_AgeOfYoungestAccount,ALL_Count,ALL_CountActive,ALL_CountClosedLast12Months,...,LoanPurpose_credit card,LoanPurpose_debt,LoanPurpose_home,LoanPurpose_other,LoanPurpose_vehicle,LoanPurpose_wedding,EmploymentType_Employed - full time,EmploymentType_Employed - part time,EmploymentType_Retired,EmploymentType_Self employed
0,4921736,03/07/2020,2000,60,0,162,17,18,12,0,...,0,0,0,1,0,0,1,0,0,0
1,1241981,04/02/2020,3000,60,0,266,30,14,10,0,...,0,0,0,1,0,0,1,0,0,0
2,5751748,02/08/2020,20000,60,0,90,52,4,2,0,...,0,0,0,0,0,0,1,0,0,0
3,7163425,23/09/2020,20000,60,0,163,19,14,6,1,...,0,0,0,0,0,0,0,0,0,1
4,227377,01/01/2020,5000,36,0,129,2,38,19,4,...,0,0,0,0,1,0,1,0,0,0


In [249]:
X = combined_df.drop(['ApplicationDate'], axis=1)

In [251]:
corr_Matrix=X.corr()
corr_Matrix

,UID,Amount,Term,Success,ALL_AgeOfOldestAccount,ALL_AgeOfYoungestAccount,ALL_Count,ALL_CountActive,ALL_CountClosedLast12Months,ALL_CountDefaultAccounts,...,LoanPurpose_credit card,LoanPurpose_debt,LoanPurpose_home,LoanPurpose_other,LoanPurpose_vehicle,LoanPurpose_wedding,EmploymentType_Employed - full time,EmploymentType_Employed - part time,EmploymentType_Retired,EmploymentType_Self employed
UID,1.000000,0.030056,-0.002103,0.042425,0.031407,0.025876,-0.000406,-0.004107,-0.020069,-0.021475,...,-0.000016,-0.009466,0.079057,-0.139635,0.101145,0.000141,0.041429,0.020608,0.020502,-0.101367
Amount,0.030056,1.000000,0.572818,-0.108800,0.183087,0.011503,-0.037423,0.014139,-0.063415,-0.276689,...,0.012589,0.012147,-0.012559,-0.097083,0.103845,0.006489,0.043751,-0.065803,-0.029279,0.024760
Term,-0.002103,0.572818,1.000000,-0.010138,0.164005,-0.001517,-0.025538,0.049809,-0.056075,-0.195510,...,-0.004811,0.010010,-0.048281,-0.073122,0.111710,0.010769,0.031589,-0.030396,-0.009148,-0.009103
Success,0.042425,-0.108800,-0.010138,1.000000,0.074819,0.001227,-0.044824,-0.064471,-0.037209,-0.139361,...,0.001969,-0.005420,0.012287,-0.017260,0.014729,-0.006575,-0.005450,0.028022,0.005503,-0.027135
ALL_AgeOfOldestAccount,0.031407,0.183087,0.164005,0.074819,1.000000,0.073303,0.187815,0.265190,0.038373,-0.036057,...,0.001094,0.026904,0.027693,-0.030479,-0.008715,0.002039,-0.070208,0.047519,0.099341,-0.003860
ALL_AgeOfYoungestAccount,0.025876,0.011503,-0.001517,0.001227,0.073303,1.000000,-0.211057,-0.272190,-0.169977,-0.015616,...,-0.003674,-0.042004,0.009512,-0.030059,0.056527,0.008226,-0.084077,0.038404,0.053411,0.056018
ALL_Count,-0.000406,-0.037423,-0.025538,-0.044824,0.187815,-0.211057,1.000000,0.640482,0.642493,0.287004,...,0.011594,0.057990,0.011308,0.037132,-0.095057,-0.013990,0.061007,-0.033777,0.001171,-0.057701
ALL_CountActive,-0.004107,0.014139,0.049809,-0.064471,0.265190,-0.272190,0.640482,1.000000,0.273038,0.326926,...,0.012460,0.087816,0.001231,0.050315,-0.123261,-0.020381,0.033841,-0.016769,0.015184,-0.042982
ALL_CountClosedLast12Months,-0.020069,-0.063415,-0.056075,-0.037209,0.038373,-0.169977,0.642493,0.273038,1.000000,0.086023,...,-0.000909,0.021326,-0.016528,0.042667,-0.048157,-0.011983,0.057519,-0.032809,-0.009375,-0.047016
ALL_CountDefaultAccounts,-0.021475,-0.276689,-0.195510,-0.139361,-0.036057,-0.015616,0.287004,0.326926,0.086023,1.000000,...,-0.013802,-0.013737,0.008202,0.009527,-0.001633,-0.005347,-0.007635,0.010081,-0.000049,0.000390


In [253]:
X = X.drop(['UID','Success'], axis = 1)

In [255]:
y = combined_df['Success']

In [259]:
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X, y)
feature_importances = xgb_model.feature_importances_

important_features = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
selected_features = important_features[important_features['Importance'] > 0.01]['Feature']

In [261]:
# sorting features importance
important_features_sorted = important_features.sort_values(by='Importance', ascending=False)
# Selecting top features with importance greater than a threshold (e.g., > 0.01)
selected_features_list = important_features_sorted[important_features_sorted['Importance'] > 0.01]
selected_features_list

,Feature,Importance
14,ALL_WorstPaymentStatusActiveAccounts,0.279722
0,Amount,0.070646
7,ALL_CountDefaultAccounts,0.049341
12,ALL_SumCurrentOutstandingBalExcMtg,0.043951
10,ALL_MeanAccountAge,0.042857
5,ALL_CountActive,0.041598
8,ALL_CountOpenedLast12Months,0.036397
11,ALL_SumCurrentOutstandingBal,0.036216
4,ALL_Count,0.035015
22,EmploymentType_Employed - full time,0.032389


In [263]:
X = X[selected_features]

In [267]:
X

,ALL_WorstPaymentStatusActiveAccounts,Amount,ALL_CountDefaultAccounts,ALL_SumCurrentOutstandingBalExcMtg,ALL_MeanAccountAge,ALL_CountActive,ALL_CountOpenedLast12Months,ALL_SumCurrentOutstandingBal,ALL_Count,EmploymentType_Employed - full time,...,LoanPurpose_vehicle,ALL_AgeOfOldestAccount,LoanPurpose_other,ALL_AgeOfYoungestAccount,ALL_CountSettled,EmploymentType_Self employed,Term,ALL_CountClosedLast12Months,LoanPurpose_debt,LoanPurpose_credit card
0,0,2000,0,15019,70.94,12,0,68555,18,1,...,0,162,1,17,6,0,60,0,0,0
1,7,3000,4,2209,104.79,10,0,2209,14,1,...,0,266,1,30,4,0,60,0,0,0
2,7,20000,1,5108,68.25,2,0,5108,4,1,...,0,90,0,52,2,0,60,0,0,0
3,0,20000,1,25738,67.50,6,0,25738,14,0,...,0,163,0,19,8,1,60,1,0,0
4,7,5000,9,5801,56.45,19,8,5801,38,1,...,1,129,0,2,19,0,36,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8842,0,5000,0,10968,52.36,16,5,10968,22,1,...,0,164,0,2,6,0,60,2,0,0
8843,7,8000,1,2204,73.57,4,0,2204,7,1,...,0,275,0,20,3,0,24,1,0,0
8844,0,1000,0,6786,44.14,13,1,6786,14,1,...,0,145,0,8,1,0,36,0,0,0
8845,0,10000,0,9398,9.25,3,3,9398,4,0,...,0,12,0,7,1,1,60,1,0,0


In [269]:
ss=StandardScaler()

In [271]:
x_scaled=ss.fit_transform(X)

In [275]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, train_size=0.9, random_state=42)

In [277]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score
import pandas as pd

def cross_val(model):
    # Ensure cross_val_score uses an appropriate scoring metric, typically accuracy for most use cases
    accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=10, scoring='accuracy')
    return accuracies.mean() * 100

def fit_evaluate(model, average='binary'):
    # Specify 'average' for multi-class problems ('macro', 'micro', 'weighted')
    name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Compute cross-validation accuracy
    cross = cross_val(model)
    
    # Compute evaluation metrics
    a_s = accuracy_score(y_test, y_pred) * 100
    pre_sc = precision_score(y_test, y_pred, average=average) * 100
    rec_sc = recall_score(y_test, y_pred, average=average) * 100
    f1_sc = f1_score(y_test, y_pred, average=average) * 100
    
    # Handle roc_auc_score differently for multi-class problems
    if len(set(y_test)) > 2:
        # Multi-class case
        roc_sc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr') * 100
    else:
        # Binary case
        roc_sc = roc_auc_score(y_test, y_pred) * 100
    
    # Construct result DataFrame
    result = pd.DataFrame(
        [[name, cross, a_s, pre_sc, rec_sc, f1_sc, roc_sc]], 
        columns=['model', 'accuracy_train_cv', 'accuracy_test', 'precision_score', 'recall_score', 'f1_score', 'roc_auc_score']
    )
    return result

In [279]:
model=XGBClassifier()
fit_evaluate(model, average='macro')

,model,accuracy_train_cv,accuracy_test,precision_score,recall_score,f1_score,roc_auc_score
0,XGBClassifier,89.399728,90.169492,69.821062,62.067505,64.635933,62.067505


In [281]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

# Balancing the dataset
X_balanced, y_balanced = smote.fit_resample(X, y)

In [285]:
# Training XGBoost model on the balanced dataset to identify important features
xgb_model_balanced = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model_balanced.fit(X_balanced, y_balanced)

# Extracting feature importances
feature_importances_balanced = xgb_model_balanced.feature_importances_

# Creating a DataFrame of features and their importance scores
important_features_balanced = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances_balanced
}).sort_values(by='Importance', ascending=False)

# Selecting top features with importance greater than a threshold (e.g., > 0.01)
selected_features_balanced = important_features_balanced[important_features_balanced['Importance'] > 0.01]['Feature']

In [287]:
selected_features_balanced

0     ALL_WorstPaymentStatusActiveAccounts
21                        LoanPurpose_debt
12                        LoanPurpose_home
13                     LoanPurpose_vehicle
18            EmploymentType_Self employed
22                 LoanPurpose_credit card
11                  EmploymentType_Retired
15                       LoanPurpose_other
1                                   Amount
19                                    Term
10     EmploymentType_Employed - part time
9      EmploymentType_Employed - full time
4                       ALL_MeanAccountAge
6              ALL_CountOpenedLast12Months
5                          ALL_CountActive
3       ALL_SumCurrentOutstandingBalExcMtg
2                 ALL_CountDefaultAccounts
Name: Feature, dtype: object

In [289]:
# Subsetting the dataset with selected features
X_balanced_selected = X_balanced[selected_features_balanced]

In [291]:
important_features_balanced.head()

,Feature,Importance
0,ALL_WorstPaymentStatusActiveAccounts,0.226666
21,LoanPurpose_debt,0.108802
12,LoanPurpose_home,0.083229
13,LoanPurpose_vehicle,0.082107
18,EmploymentType_Self employed,0.068823


In [293]:
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(
    X_balanced_selected, y_balanced, test_size=0.2, random_state=42
)

In [295]:
X_train = X_train_balanced
X_test = X_test_balanced
y_train = y_train_balanced
y_test = y_test_balanced

In [297]:
model=XGBClassifier()
fit_evaluate(model, average='macro')

,model,accuracy_train_cv,accuracy_test,precision_score,recall_score,f1_score,roc_auc_score
0,XGBClassifier,93.123028,93.249211,93.343843,93.241979,93.244691,93.241979
